docker exec rag-mcq-airflow airflow dags unpause pdf_ingestion_dag

docker exec rag-mcq-airflow airflow dags trigger pdf_ingestion_dag

In [ ]:
import os

# Ghi đè connection string để dùng localhost thay vì postgres
os.environ['POSTGRES_HOST'] = 'localhost'

In [ ]:
!curl -X DELETE "http://localhost:9200/mcq-documents" -u admin:admin

### Bước 0

In [ ]:
import torch
# torch.cuda.empty_cache()
print(torch.cuda.is_available())

In [ ]:
from src.models.document import Base
from src.db.session import engine

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("""
        DO
        $$
        DECLARE
            r RECORD;
        BEGIN
            FOR r IN (SELECT tablename FROM pg_tables WHERE schemaname = 'public') LOOP
                EXECUTE 'DROP TABLE IF EXISTS ' || quote_ident(r.tablename) || ' CASCADE';
            END LOOP;
        END
        $$;
    """))
    conn.commit()

print("✅ Dropped all tables successfully — fresh start!")

In [ ]:
Base.metadata.create_all(bind=engine)

### Bước 1

In [ ]:
from pathlib import Path

API_BASE_URL = "http://localhost:8000"
AIRFLOW_URL = "http://localhost:8080"

# Host & port các service
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432          # hoặc 5433 nếu muốn connect airflow-db
REDIS_HOST = "localhost"
REDIS_PORT = 6379
OPENSEARCH_HOST = "http://localhost:9200"
OLLAMA_HOST = "http://localhost:11434"

# Cấu hình RAG
TOP_K = 5
USE_HYBRID = True
TIMEOUT = 60

In [ ]:
from src.config import get_settings

settings = get_settings()
pdf_dir = Path(settings.data.pdf_dir)

pdf_files = list(pdf_dir.glob("*.pdf"))
pdf_paths = [str(f) for f in pdf_files]

print(f"Found {len(pdf_paths)} PDF files:")
pdf_paths

### Bước 2 

In [ ]:
def sanitize_metadata(metadata: dict):
    safe = {}
    for k, v in metadata.items():
        if callable(v):
            try:
                safe[k] = v()  # gọi method nếu cần
            except Exception:
                safe[k] = None
        else:
            safe[k] = v
    return safe

In [ ]:
import sys
import os
import locale
import io

def calculate_optimal_workers() -> int:
    """Auto-calculate optimal workers for current system."""
    cpu_count = os.cpu_count() or 1
    return max(3, int(cpu_count * 0.7))

# Auto-calculate
optimal_workers = calculate_optimal_workers()

# optimal_workers = 7
print(optimal_workers)

In [ ]:
from src.db.session import get_db_context
from src.models.document import Document
from src.services.factories import make_pdf_parser
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def _parse_and_save_single_pdf(args: tuple) -> tuple:
    """
    Parse a single PDF and save to database (for parallel execution).

    Args:
        args: Tuple of (pdf_path, processed_count, total_count)

    Returns:
        tuple: (doc_id, success, error_message)
    """
    pdf_path, processed_count, total_count = args
    doc_id = Path(pdf_path).stem

    try:
        parser = make_pdf_parser()

        # Parse PDF
        parsed = parser.parse_pdf(pdf_path) # Write pdf to markdown

        # Check if parsing failed
        if parsed is None:
            return (doc_id, False, "Parsing returned None")

        safe_metadata = sanitize_metadata(parsed["metadata"])

        # Create document object
        document = Document(
            doc_id=doc_id,
            filename=safe_metadata.get("file_name", ""),
            file_path=pdf_path,
            title=doc_id + parsed.get("sections", {})[3]['title'],
            full_text=doc_id + ' - ' + parsed.get("sections", {})[3]['title'] + '\n' + parsed.get("full_text", ""),
            raw_content=parsed.get("full_text", ""),
            page_count=safe_metadata.get("page_count", 0),
            sections=parsed.get("sections", {}),
            tables=parsed.get("tables", {}),
            doc_metadata=safe_metadata,
            source_folder=Path(pdf_path).parent.name,
            processing_status="completed",
        )

        # Save to DB
        with get_db_context() as db:
            # Check if already exists
            existing = db.query(Document).filter(Document.doc_id == doc_id).first()
            if existing:
                return (doc_id, False, "Already exists")

            db.add(document)
            db.commit()

        return (doc_id, True, None)

    except Exception as e:
        return (doc_id, False, str(e))


def extract_pdfs_debug(pdf_paths: list, max_workers: int = 4):
    """
    Extract and save PDFs to database with parallel processing.

    Args:
        pdf_paths: List of PDF file paths to process
        max_workers: Number of parallel workers (default: 4)
    """
    processed = 0
    errors = 0
    doc_ids = []
    failed_docs = []

    total = len(pdf_paths)

    print(f"\n{'='*80}")
    print(f"🚀 Starting parallel PDF extraction")
    print(f"📁 Total PDFs to process: {total}")
    print(f"⚙️  Workers: {max_workers}")
    print(f"{'='*80}\n")

    start_time = time.time()

    # Prepare task arguments
    tasks = [(pdf_path, i+1, total) for i, pdf_path in enumerate(pdf_paths)]
    print('PATH:', tasks)
    print(Path('data\\pdf\\Public_427.pdf').stem)
    # Process PDFs in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(_parse_and_save_single_pdf, task): task[0]
            for task in tasks
        }

        completed_count = 0
        for future in as_completed(futures):
            pdf_path = futures[future]
            doc_id = Path(pdf_path).stem

            try:
                result = future.result(timeout=300)  # 5 minutes per PDF
                doc_id_result, success, error = result

                completed_count += 1
                progress = (completed_count / total) * 100

                if success:
                    doc_ids.append(doc_id_result)
                    processed += 1
                    print(f"[{completed_count}/{total}] ({progress:.1f}%) ✅ {doc_id_result}")
                else:
                    errors += 1
                    failed_docs.append((doc_id_result, error))
                    if error == "Already exists":
                        print(f"[{completed_count}/{total}] ({progress:.1f}%) ⏩ {doc_id_result} - Already exists")
                    else:
                        print(f"[{completed_count}/{total}] ({progress:.1f}%) ❌ {doc_id_result} - {error[:50]}")

            except Exception as e:
                completed_count += 1
                errors += 1
                failed_docs.append((doc_id, str(e)))
                progress = (completed_count / total) * 100
                print(f"[{completed_count}/{total}] ({progress:.1f}%) ⏱️  {doc_id} - Timeout/Error: {str(e)[:50]}")

    total_time = time.time() - start_time

    # Print summary
    print(f"\n{'='*80}")
    print(f"📊 EXTRACTION SUMMARY")
    print(f"{'='*80}")
    print(f"✅ Successfully processed: {processed}")
    print(f"⏩ Already existed: {len([d for d in failed_docs if 'Already exists' in str(d[1])])}")
    print(f"❌ Errors: {errors}")
    print(f"⏱️  Total time: {total_time:.1f}s")
    print(f"⚡ Speed: {total/total_time:.2f} PDF/s")

    if failed_docs:
        print(f"\n⚠️  Failed documents:")
        for doc_id, error in failed_docs:
            if error != "Already exists":
                print(f"  • {doc_id}: {error[:80]}")

    print(f"{'='*80}\n")

    return {"processed": processed, "errors": errors, "doc_ids": doc_ids, "total_time": total_time}

In [ ]:
extract_result = extract_pdfs_debug(pdf_paths, max_workers=7)

### Bước 3

In [ ]:
from pathlib import Path
from src.db.session import get_db_context
from src.models.document import Document, DocumentChunk
from src.services.factories import make_text_chunker
import re

def chunk_documents_from_markdown(markdown_dir: str = 'private_test_output_html'):
    """
    Chunk documents directly from markdown files in a folder.

    Args:
        markdown_dir: Directory containing markdown files to chunk

    Returns:
        Dictionary with chunking statistics
    """
    chunker = make_text_chunker()
    total_chunks = 0
    processed = 0
    chunk_ids = []

    # Get all markdown files
    md_path = Path(markdown_dir)
    md_files = list(md_path.glob('*.md'))

    if not md_files:
        print(f"❌ No markdown files found in {markdown_dir}")
        return {"processed": 0, "chunks_created": 0, "chunk_ids": []}

    print(f"ℹ Found {len(md_files)} markdown files to chunk")

    with get_db_context() as db:
        for md_file in md_files:
            doc_id = md_file.stem  # e.g., "Public_427" from "Public_427.md"

            try:
                # Read markdown file
                content = md_file.read_text(encoding='utf-8')

                # Extract title from markdown (first ## header)
                title_match = re.search(r'^##\s+(.+)$', content, re.MULTILINE)
                title = title_match.group(1).strip() if title_match else doc_id

                # Prepare document data
                doc_data = {
                    "title": title,
                    "full_text": content,
                    "sections": [],
                }

                # Chunk the document
                chunks = chunker.chunk_document(doc_data, doc_id)

                if not chunks:
                    print(f"⚠ No chunks created for document {doc_id}")
                    continue

                # Check if document exists in DB
                document = db.query(Document).filter(Document.doc_id == doc_id).first()

                # Create or use existing document
                if not document:
                    document = Document(
                        doc_id=doc_id,
                        filename=md_file.name,
                        file_path=str(md_file),
                        title=title,
                        full_text=content,
                        raw_content=content,
                        page_count=len(content.split('\n')),  # Approximate
                        sections={},
                        tables={},
                        doc_metadata={"source": "markdown"},
                        source_folder="private_test_output_html",
                        processing_status="completed",
                    )
                    db.add(document)
                    db.flush()  # Get the ID without committing

                # Add chunks to database
                for chunk in chunks:
                    db_chunk = DocumentChunk(
                        chunk_id=chunk["chunk_id"],
                        document_id=document.id,
                        document_file_name=document.doc_id,
                        document_title=document.title,
                        chunk_text=document.title + "\n" + chunk["chunk_text"],
                        chunk_index=chunk["chunk_index"],
                        section_name=chunk.get("section_name", "markdown"),
                        chunk_type=chunk.get("chunk_type", "text"),
                        word_count=chunk.get("word_count", 0),
                        char_count=chunk.get("char_count", 0),
                        chunk_metadata=chunk,
                        embedding_status="pending",
                        indexed_in_opensearch="pending",
                    )
                    db.add(db_chunk)
                    chunk_ids.append(chunk["chunk_id"])

                processed += 1
                total_chunks += len(chunks)
                print(f"✅ Chunked document {doc_id}: {len(chunks)} chunks")

            except Exception as e:
                print(f"❌ Error chunking document {doc_id}: {e}")
                continue

        # Commit all changes at the end
        db.commit()

    print(f"\n📊 Summary: processed={processed}, chunks_created={total_chunks}, chunk_ids={len(chunk_ids)}")
    return {"processed": processed, "chunks_created": total_chunks, "chunk_ids": chunk_ids}

In [ ]:
chunk_result = chunk_documents_from_markdown()
print(chunk_result)

### TODO: Extract Title của file PDF. Làm sao để 

In [113]:
sections = [{"title":"1. BÀN DI CHU Ộ T","level":3,"content":""},{"title":"Cài đặ t Bàn di chu ộ t","level":3,"content":"-Để điề u ch ỉnh cài đặ t và c ử ch ỉ c ủ a bàn di chu ộ t, ho ặc để t ắ t bàn di chu ộ t:\n\n-Nh ập touchpad settings (cài đặ t bàn di chu ộ t) vào ô tìm ki ế m trên thanh tác v ụ , sau đó nhấ n enter.\n\n-Ch ọ n m ột cài đặ t.\n\n"},{"title":"Để b ậ t bàn di chu ộ t","level":3,"content":"-Nh ập touchpad settings (cài đặ t bàn di chu ộ t) vào ô tìm ki ế m trên thanh tác v ụ , sau đó nhấ n enter.\n\n-S ử d ụ ng chu ộ t ngoài nh ấ p vào nút touchpad (bàn di chu ộ t).\n\n-ho ặ c -\n\nNh ấ n phím Tab nhi ề u l ần đế n khi con tr ỏ n ằ m trên nút touchpad (bàn di chu ộ t). Sau đó, nhấn phím cách để ch ọ n nút.\n\n"},{"title":"VIETTEL AI RACE MÔ T Ả VÙNG BÀN PHÍM MÁY TÍNH","level":3,"content":""},{"title":"HP","level":3,"content":"TD655\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE MÔ T Ả VÙNG BÀN PHÍM MÁY TÍNH HP","level":3,"content":"TD655\n\nL ầ n ban hành: 1\n\n"},{"title":"2. ĐÈN","level":3,"content":""},{"title":"VIETTEL AI RACE MÔ T Ả VÙNG BÀN PHÍM MÁY TÍNH HP","level":3,"content":"TD655\n\nL ầ n ban hành: 1\n\n"},{"title":"3. NÚT","level":3,"content":"TD655\n\n"},{"title":"4. PHÍM ĐẶ C BI Ệ T","level":3,"content":""},{"title":"VIETTEL AI RACE MÔ T Ả VÙNG BÀN PHÍM MÁY TÍNH","level":3,"content":""},{"title":"HP","level":3,"content":"độ Ng ủ (ch ỉ m ộ t s ố s ả n ph ẩ m).\n\nKhi máy tính đang ở tr ạ ng thái Ng ủ đông, nhấn nhanh nút này để thoát ch ế độ Ng ủ đông.\n\nQUAN TR Ọ NG: Nh ấ n và gi ữ nút ngu ồ n s ẽ làm m ấ t nhfing thông tin chưa lưu.\n\nN ế u máy tính ng ừ ng ph ả n h ồ i và quy trình t ắ t máy vô hi ệ u, nh ấ n và nh ấ n gifi nút ngu ồ n trong ít nh ất 10 giây để t ắ t máy tính.\n\nĐể tìm hi ể u thêm v ề các cài đặ t ngu ồn điệ n, hãy xem các tùy ch ọ n ngu ồn điệ n c ủ a b ạ n:\n\nNh ấ p chu ộ t ph ả i vào bi ểu tượ ng Power (Ngu ồ n) , r ồi sau đó ch ọ n Power Options (Tùy ch ọ n Ngu ồn điệ n).\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE MÔ T Ả VÙNG BÀN PHÍM MÁY TÍNH HP","level":3,"content":"TD655\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE MÔ T Ả VÙNG BÀN PHÍM MÁY TÍNH HP","level":3,"content":"TD655\n\nL ầ n ban hành: 1\n\n"},{"title":"5. PHÍM THAO TÁC","level":3,"content":"Các phím thao tác s ẽ th ự c thi các ch ức năng hệ th ố ng sfi d ụng thường xuyên như được định nghĩa bở i các ký hi ệ u bi ểu tượ ng trên các phím ch ức năng t ừ f1 đế n f12. Các phím thao tác này s ẽ khác nhau tùy theo máy tính.\n\nĐể s ử d ụ ng m ộ t phím thao tác, nh ấ n fn, r ồ i nh ấ n m ột trong các phím đượ c li ệ t kê trong b ả ng sau.\n\nTD655\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE MÔ T Ả VÙNG BÀN PHÍM MÁY TÍNH","level":3,"content":""},{"title":"HP","level":3,"content":""}]
print(sections[3]['title'])

sections = [{"title":"Introduction","level":1,"content":"TD004\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"Hướ ng d ẫ n t ự l ắp Điệ n M ặ t Tr ờ i","level":3,"content":"Trướ c khi l ắp đặt, ngườ i  thi  công  s ẽ xác nh ận đị a hình, v ị trí,  hướ ng và góc nghiêng theo b ả n thi ế t k ế v ới đị a hình th ự c t ế m ộ t l ầ n n ữa. Sau khi xác đị nh v ị trí l ắp đặ t t ừ ng b ộ ph ậ n s ẽ b ắ t tay vào thi công.\n\n"},{"title":"1 Các bướ c l ắp Điệ n M ặ t tr ờ i","level":3,"content":""},{"title":"1.1 L ắp đặt khung giá đỡ giàn pin","level":3,"content":"M ục đích: Khung giá đỡ gi ữ c ố đị nh các t ấ m pin m ặ t tr ời, đả m b ảo độ nghiêng và hướ ng l ắp đặ t t ối ưu để thu được lượ ng ánh sáng nhi ề u nh ấ t.\n\n"},{"title":"· Phương án lắp đặ t:","level":3,"content":"o Áp mái: L ắ p tr ự c ti ế p trên mái nhà, chi phí th ấ p, ti ế t ki ệ m di ệ n tích.\n\no Giá đỡ độ c l ậ p: L ắ p trên khung giàn b ằng thép/nhôm đặ t ở sân, vườ n ho ặ c mái b ằng. Phương án này tốn kém hơn do phả i thi công khung, xà g ồ và yêu c ầ u k ế t c ấ u ch ị u t ả i v ữ ng ch ắ c, ch ố ng gió bão.\n\nLưu ý: C ầ n kh ảo sát địa hình, độ d ố c mái, v ậ t li ệu mái (tôn, bê tông, ngói…) để ch ọ n ph ụ ki ệ n liên k ế t phù h ợ p.\n\n"},{"title":"1.2 G ắ n t ấm pin lên giá đỡ","level":3,"content":"M ục đích: C ố đị nh các t ấ m pin vào khung, t ạo điề u ki ệ n cho vi ệ c h ấ p th ụ ánh sáng m ặ t tr ờ i hi ệ u qu ả .\n\n"},{"title":"· Kho ả ng cách k ỹ thu ậ t:","level":3,"content":"o Gi ữ a các t ấ m pin: 10mm để tránh giãn n ở nhi ệ t và thu ậ n ti ệ n cho thoát nướ c.\n\no Gi ữ a t ấ m pin và mái: ≥100mm để t ạo không gian lưu thông gió, giúp tả n nhi ệt và tăng tuổ i th ọ pin.\n\nLưu ý: S ử d ụ ng bulong, k ẹ p pin và ph ụ ki ện chính hãng để đả m b ả o an toàn, tránh rung l ắ c khi có gió m ạ nh.\n\n"},{"title":"1.3 K ế t n ố i t ấ m pin","level":3,"content":"M ục đích: Liên k ế t các t ấ m pin thành m ạch điện (string) để t ạo điệ n áp và dòng điệ n phù h ợ p cho inverter.\n\n"},{"title":"· Cách th ự c hi ệ n:","level":3,"content":"o K ế t n ố i n ố i ti ế p các t ấ m pin b ằ ng dây chuyên d ụ ng (MC4).\n\n1\n\nTD004\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"Hướ ng d ẫ n t ự l ắp Điệ n M ặ t Tr ờ i","level":3,"content":"o Tùy theo thi ế t k ế , các string s ẽ đượ c gom v ề h ộp combiner box trướ c khi đưa vào inverter.\n\n"},{"title":"· Lưu ý:","level":3,"content":"o B ố trí string theo hướng mái để gi ả m t ổ n th ấ t do che bóng.\n\no Dây d ẫ n c ầ n có ti ế t di ện đủ l ớ n, ch ố ng tia UV và ch ố ng th ấ m.\n\n"},{"title":"1.4 L ắp đặ t bi ế n t ầ n (Inverter)","level":3,"content":"M ục đích: Bi ến đổi dòng điệ n DC t ừ t ấ m pin thành AC s ử d ụ ng cho thi ế t b ị điện trong gia đình/doanh nghiệ p.\n\n"},{"title":"· Yêu c ầ u v ị trí:","level":3,"content":"o Cao ráo, thoáng mát, tránh mưa trự c ti ế p, ẩm ướ t, nhi ệt độ cao.\n\no Có kho ả ng tr ống để d ễ dàng quan sát màn hình hi ể n th ị và thao tác khi c ầ n thi ế t.\n\nLưu ý: Tránh l ắ p g ầ n b ế p, bình gas ho ặ c khu v ực có nguy cơ cháy nổ .\n\n"},{"title":"1.5 L ắp đặ t thi ế t b ị ch ống phát ngược lướ i","level":3,"content":"M ục đích: Đả m b ả o h ệ th ố ng ch ỉ bám t ải và không phát điện ngược lên lướ i điệ n qu ố c gia (n ếu không đượ c phép).\n\n"},{"title":"· Cách th ự c hi ệ n:","level":3,"content":"o G ắ n CT (Current Transformer) để đo dòng điệ n.\n\no K ế t n ố i Meter để g ử i tín hi ệ u v ề inverter.\n\nLưu ý: Bướ c này r ấ t quan tr ọng để h ệ th ố ng v ậ n hành an toàn, tránh vi ph ạ m quy đị nh c ủa ngành điệ n.\n\n"},{"title":"1.6 C ấ p ngu ồn PV, AC, pin lưu trữ v ớ i bi ế n t ầ n","level":3,"content":"M ục đích: Kh ởi độ ng h ệ th ố ng và ki ể m tra kh ả năng hoạt độ ng c ủ a các thành ph ầ n.\n\n"},{"title":"· Trình t ự th ự c hi ệ n:","level":3,"content":"Đả m b ả o t ấ t c ả CB và công t ắ c ở tr ạ ng thái OFF .\n\nB ậ t công t ắ c ON trên inverter .\n\n2\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"Hướ ng d ẫ n t ự l ắp Điệ n M ặ t Tr ờ i","level":3,"content":"TD004\n\nL ầ n ban hành: 1\n\nB ậ t CB DC (t ừ pin v ề inverter), ch ờ kho ả ng 60 giây để inverter nh ậ n tín hi ệ u.\n\nB ậ t CB AC (t ừ inverter ra t ải/lướ i).\n\nN ếu có pin lưu trữ , b ậ t CB Battery cu ố i cùng.\n\nLưu ý: Luôn để kho ả ng ngh ỉ 1 -2 phút gi ữ a các l ầ n c ấ p ngu ồ n để tránh s ố c điệ n cho thi ế t b ị .\n\n"},{"title":"1.7 K ế t n ố i Wifi v ớ i bi ế n t ầ n","level":3,"content":"M ục đích: Giúp giám sát s ản lượ ng và tình tr ạ ng ho ạt độ ng c ủ a h ệ th ố ng qua điệ n tho ạ i.\n\n"},{"title":"· Cách th ự c hi ệ n:","level":3,"content":"o Tùy theo hãng inverter (Huawei, Sungrow, Growatt, Solis…), cài đặ t App chuyên d ụ ng.\n\no K ế t n ối Wifi theo hướ ng d ẫ n t ừ nhà s ả n xu ấ t.\n\nLưu ý: Các nhà phân ph ối (như DHC Solar) thườ ng h ỗ tr ợ k ỹ thu ậ t t ừ xa trong bướ c này.\n\n"},{"title":"1.8 V ậ n hành, b ả o trì","level":3,"content":"Theo dõi v ậ n hành:\n\no Qua App, ngườ i dùng có th ể bi ế t: s ản lượng điệ n, công su ấ t tiêu th ụ, điệ n bù lướ i, ph ần trăm pin lưu trữ , c ả nh báo l ỗ i.\n\n"},{"title":"· B ảo trì đị nh k ỳ :","level":3,"content":"o V ệ sinh b ề m ặ t t ấ m pin b ằng nướ c s ạch, khăn mề m ho ặ c d ụ ng c ụ chuyên d ụ ng (tránh dùng hóa ch ấ t m ạ nh).\n\no C ắ t t ỉ a cây c ố i, d ọ n lá r ụng để không che bóng t ấ m pin.\n\no Ki ể m tra dây d ẫn, CB, inverter đị nh k ỳ 6 -12 tháng/l ầ n.\n\n"},{"title":"2 Lưu ý khi t ự l ắp điện năng lượ ng m ặ t tr ờ i","level":3,"content":"Lưu ý 1 : Ch ỉ c ầ n m ộ t t ấ m pin m ặ t tr ờ i b ị bóng râm che bóng có th ể khi ế n c ả h ệ th ố ng điệ n m ặ t  tr ờ i  gi ả m hi ệ u su ất.  Đồ ng th ờ i  c ần đả m b ả o r ằ ng n ếu mùa thay đổi thì cũng không xu ấ t hi ện bóng râm trong tương lai.\n\n3\n\nTD004\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"Hướ ng d ẫ n t ự l ắp Điệ n M ặ t Tr ờ i","level":3,"content":"L ầ n ban hành: 1\n\nLưu ý 2 : N ế u di ệ n tích mái r ộ ng khi l ắp đặ t các t ấ m pin m ặ t tr ờ i nên l ắ p 1 ví tr ị nh ấ t đị nh, ch ừ a ra m ộ t  kho ảng không gian đáng kể để có  th ể b ổ sung thêm nhi ề u t ấ m pin năng lượ ng m ặ t tr ời trong tương lai.\n\nLưu ý 3 : C ầ n có l ối đi lại để có th ể v ệ sinh t ấ m pin, b ả o trì, b ảo dưỡng đị nh k ỳ .\n\nLưu ý 4 : Đả m b ả o các dây d ẫ n c ần được cách điệ n và ch ố ng th ấm. Để tránh điệ n gi ậ t ta cũng cầ n k ế t n ố i dây ti ếp đấ t.\n\nLưu ý 5 : Khung giá đỡ t ấ m pin theo thi ế t k ế k ỹ thu ậ t. Không nên t ự ch ế các lo ạ i k ẹ p, giá đỡ không đúng kỹ thu ậ t.\n\nLưu ý 6 : Hãy th ự c hi ện theo các hướ ng d ẫn được đi kèm thiế t b ị c ủ a nhà s ả n xu ấ t.\n\n"},{"title":"3 B ảng so sánh Điệ n m ặ t tr ời và Điện lưới nhà nướ c","level":3,"content":"4\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"Hướ ng d ẫ n t ự l ắp Điệ n M ặ t Tr ờ i","level":3,"content":"TD004\n\nL ầ n ban hành: 1\n\nTiêu chí\n\n"},{"title":"Điệ n m ặ t tr ờ i","level":3,"content":""},{"title":"Điện lưới nhà nướ c (EVN)","level":3,"content":"ti ệ n và ổn định hơn trong ngắ n h ạ n, song chi phí ph ụ thu ộ c vào giá điệ n và ít mang tính ch ủ độ ng.\n\n5\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"Hướ ng d ẫ n t ự l ắp Điệ n M ặ t Tr ờ i","level":3,"content":"TD004\n\nL ầ n ban hành: 1\n\nTiêu chí\n\nĐiệ n m ặ t tr ờ i\n\nĐiện lưới nhà nướ\n\nc (EVN)\n\n6\n\n"}]
print(sections[3]['title'])

sections = [{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\nCẤU HÌNH VÀ THÔNG SỐ KỸ THUẬT CHI TIẾT CỦA MÁY SIÊU ÂM ICU ĐO ĐƯỢC HUYẾT ĐỘNG VÀ MÁY HẠ THÂN NHIỆT CHỈ HUY\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY","level":3,"content":"TD636\n\nL ầ n ban hành: 1\n\n"}]
print(sections[3]['title'])

# for section in sections:
#     print(f"Original: {section['title']}")

# sections = [{"title":"VIETTEL AI RACE MÀN HÌNH BIỂU ĐỒ SẢN XUẤT CỦA","level":3,"content":"TD662\n\n"},{"title":"MÀN HÌNH CẢM ỨNG E-MULTI","level":3,"content":"Lần ban hành: 1\n\n"},{"title":"Các thành phần trên màn hình Biểu đồ sản xuất","level":3,"content":""},{"title":"Các thành phần trên màn hình","level":3,"content":""},{"title":"Mô tả","level":3,"content":"Màn hình này hiển thị biểu đồ của một biến được chọn. Tên biến và các giá trị hiện tại được trình bày bên dưới biểu đồ. Có thể sử dụng các chức năng sau đây:\n\nĐịnh nghĩa biểu đồ tham chiếu\n\nHiển thị biểu đồ xu hướng cuối cùng\n\nTheo dõi bằng miền dung sai đã chọn Điểm chuyển tiếp (là điểm mà tại đó hệ thống thay đổi từ áp suất phun sang áp suất giữ) hiển thị dưới dạng đường dọc màu ngọc lam. Phạm vi điểm chuyển tiếp hiển thị dưới dạng thanh màu trắng ở cạnh trên của sơ đồ.\n\nTrong quy trình thiết lập đúng cách, thanh này sẽ rất hẹp. Giá trị trung bình của tất cả các điểm chuyển tiếp hiển thị dưới dạng đường màu đen trong thanh màu trắng. Có thể đặt chế độ hiển thị là:\n\nThời gian (biểu đồ y/t)\n\nVị trí (biểu đồ y/x)\n\nPhân tách (dạng hỗn hợp, cả hai loại biểu đồ)\n\nTD662\n\nLần ban hành: 1\n\n"},{"title":"VIETTEL AI RACE MÀN HÌNH BIỂU ĐỒ SẢN XUẤT CỦA","level":3,"content":""},{"title":"MÀN HÌNH CẢM ỨNG E-MULTI","level":3,"content":""},{"title":"· Các nút menu phía dưới","level":3,"content":""},{"title":"VIETTEL AI RACE MÀN HÌNH BIỂU ĐỒ SẢN XUẤT CỦA MÀN HÌNH CẢM ỨNG E-MULTI","level":3,"content":"TD662\n\nLần ban hành: 1\n\n"},{"title":"VIETTEL AI RACE MÀN HÌNH BIỂU ĐỒ SẢN XUẤT CỦA MÀN HÌNH CẢM ỨNG E-MULTI","level":3,"content":"TD662\n\nLần ban hành: 1\n\nTD662\n\nLần ban hành: 1\n\n"},{"title":"VIETTEL AI RACE","level":3,"content":""},{"title":"MÀN HÌNH BIỂU ĐỒ SẢN XUẤT CỦA MÀN HÌNH CẢM ỨNG E-MULTI","level":3,"content":""}]
# for section in sections:
#     print(f"Original: {section['title']}")

VIETTEL AI RACE MÔ T Ả VÙNG BÀN PHÍM MÁY TÍNH
1 Các bướ c l ắp Điệ n M ặ t tr ờ i
C Ấ U HÌNH VÀ THÔNG S Ố K Ỹ THU Ậ T CHI TI Ế T C Ủ A MÁY SIÊU ÂM ICU ĐO ĐƯỢ C HUY ẾT ĐỘ NG VÀ MÁY H Ạ THÂN NHI Ệ T CH Ỉ HUY


### Bước 4

In [ ]:
import os
from opensearchpy import OpenSearch
from opensearchpy.helpers import bulk
from src.services.factories import make_embeddings_service

# --- Lấy thông số từ môi trường ---
OPENSEARCH_URL = os.environ.get("OPENSEARCH_URL", "http://localhost:9200")
OPENSEARCH_INDEX = os.environ.get("OPENSEARCH_INDEX", "mcq-documents")
EMBEDDING_DIM = int(os.environ.get("EMBEDDING_DIM", 768))

# --- Tạo client OpenSearch ---
client = OpenSearch(OPENSEARCH_URL)

# --- Xóa index cũ nếu có ---
client.indices.delete(index=OPENSEARCH_INDEX, ignore=[400, 404])

# --- Tạo index mới với mapping đúng cho OpenSearch k-NN ---
client.indices.create(
    index=OPENSEARCH_INDEX,
    body={
        "settings": {
            "index": {
                "knn": True,  # Bật k-NN plugin
                "knn.algo_param.ef_search": 100
            }
        },
        "mappings": {
            "properties": {
                "chunk_id": {"type": "keyword"},
                "document_id": {"type": "keyword"},
                "chunk_text": {"type": "text"},
                "embedding": {
                    "type": "knn_vector",  # Đổi từ dense_vector sang knn_vector
                    "dimension": EMBEDDING_DIM,  # Đổi từ dims sang dimension
                    "method": {
                        "name": "hnsw",
                        "space_type": "l2",
                        "engine": "nmslib",
                        "parameters": {
                            "ef_construction": 128,
                            "m": 24
                        }
                    }
                },
                "chunk_index": {"type": "integer"},
                "section_name": {"type": "keyword"},
                "chunk_type": {"type": "keyword"},
            }
        }
    }
)

print(f"✅ Index '{OPENSEARCH_INDEX}' created successfully!")

In [ ]:
# --- Hàm generate embeddings và bulk index ---
async def generate_embeddings_and_index(chunk_ids: list):
    embeddings_service = make_embeddings_service()

    with get_db_context() as db:
        chunks = db.query(DocumentChunk).filter(DocumentChunk.chunk_id.in_(chunk_ids)).all()
        if not chunks:
            print("No chunks found")
            return {"processed": 0, "indexed": 0}

        texts = [chunk.chunk_text for chunk in chunks]
        embeddings = await embeddings_service.embed_texts(texts)

        bulk_ops = []
        for chunk, embedding in zip(chunks, embeddings):
            chunk.embedding_status = "completed"
            chunk.embedding = embedding.tolist()
            chunk.chunk_metadata = chunk.chunk_metadata or {}
            chunk.chunk_metadata["embedding"] = embedding.tolist()

            # ✅ Access via relationship (already loaded)
            doc = chunk.document  # No extra query needed!

            bulk_ops.append({
                "_op_type": "index",
                "_index": OPENSEARCH_INDEX,
                "_id": chunk.chunk_id,
                "_source": {
                    "chunk_id": chunk.chunk_id,
                    "document_id": chunk.document_id,
                    "chunk_index": chunk.chunk_index,
                    "chunk_text": chunk.chunk_text,
                    "section_name": chunk.section_name,
                    "chunk_type": chunk.chunk_type,
                    "embedding": chunk.embedding,
                    "document_file_name": doc.filename if doc else None,
                    "document_title": doc.title if doc else None
                }
            })

        db.commit()

        success, _ = bulk(client, bulk_ops, refresh=True)
        print(f"✅ Indexed {success}/{len(chunks)} chunks")

    return {"processed": len(chunks), "indexed": success}

In [ ]:
embed_index_result = await generate_embeddings_and_index(chunk_result["chunk_ids"])
print(embed_index_result)

### Bước 6

In [ ]:
"Mục đích của từ ""luôn luôn"" và ""nào đó"", theo danh sách thẩm định thuật ngữ trong tài liệu Public_061, là hàm ý tính không mơ hồ và tính kiểm chứng của đặc tả?","Tính nhập nhằng, tính thiếu định lượng","Tính tuyệt đối, tính thuyết phục","Tính tuyệt đối, tính nhập nhằng","Tính thuyết phục, tính nhập nhằng"

In [ ]:
query="""
Đâu không phải nguyên tắc cốt lõi của khảo sát đặc tả mức cao theo tài liệu Public_061?
- Hãy là khách hàng của sản phẩm",
- Hãy nghiên cứu các chuẩn và hướng dẫn hiện hành",
- Hãy xem xét và kiểm thử các phần mềm tương tự",
- Hãy phối hợp với đội nhóm và khách hàng"
"""

In [ ]:
query="""
Mục đích của từ ""luôn luôn"" và ""nào đó"", theo danh sách thẩm định thuật ngữ trong tài liệu Public_061, là hàm ý tính không mơ hồ và tính kiểm chứng của đặc tả?
- Tính nhập nhằng, tính thiếu định lượng
- Tính tuyệt đối, tính thuyết phục
- Tính tuyệt đối, tính nhập nhằng
- Tính thuyết phục, tính nhập nhằng
"""

In [ ]:
query="""
Một trong những đặc điểm khiến Logic bomb nguy hiểm là gì?
- Luôn xuất hiện dưới dạng file thực thi .exe
- Dễ dàng phát hiện bằng mắt thường
- Có khả năng ẩn trong mã nguồn phần mềm hợp pháp
- Không thể kích hoạt bằng sự kiện người dùng
"""

In [ ]:
import requests
url = "http://localhost:8000/api/v1/search/"
payload = {
    "query": query,
    "top_k": 30,
    "use_hybrid": True
}
r = requests.post(url, json=payload, timeout=30)
print(r.status_code)

In [ ]:
from pprint import pprint
pprint(r.json())

In [ ]:
# /Users/dohainam/test_rag_ask_api.py
import requests
import json
import os
from dotenv import load_dotenv

# 1. Tải biến môi trường (nếu có)
load_dotenv()

# Cấu hình API của bạn
# Đảm bảo biến môi trường API_BASE_URL được đặt, hoặc sử dụng mặc định
API_BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")
ASK_API_ENDPOINT = f"{API_BASE_URL}/api/v1/ask"
TIMEOUT = 300 # Timeout tính bằng giây, tăng nếu LLM mất nhiều thời gian

print(f"Kiểm tra API RAG tại: {ASK_API_ENDPOINT}\n")

# 2. Chuẩn bị câu hỏi mẫu
# THAY ĐỔI CÂU HỎI, TÙY CHỌN VÀ source_folder (nếu có) CHO PHÙ HỢP VỚI DỮ LIỆU CỦA BẠN
question_to_ask = "Đâu không phải nguyên tắc cốt lõi của khảo sát đặc tả mức cao theo tài liệu Public_061?"
options_for_question = {
    "A": "Hãy là khách hàng của sản phẩm",
    "B": "Hãy nghiên cứu các chuẩn và hướng dẫn hiện hành",
    "C": "Hãy xem xét và kiểm thử các phần mềm tương tự",
    "D": "Hãy phối hợp với đội nhóm và khách hàng"
}
# Nếu bạn muốn lọc theo source_folder, hãy bỏ comment dòng dưới và thay đổi giá trị
# source_folder_filter = "my_research_papers" # THAY ĐỔI THÀNH source_folder CỦA BẠN

payload = {
    "question": question_to_ask,
    "options": options_for_question,
    "top_k": 5,           # Số lượng chunks để lấy về từ OpenSearch
    "use_hybrid": True    # Sử dụng tìm kiếm kết hợp (BM25 + Vector + RRF)
}

# Thêm source_folder nếu được định nghĩa
# if 'source_folder_filter' in locals():
#     payload['source_folder'] = source_folder_filter

print("Payload gửi đi:")
pprint(json.dumps(payload, indent=2, ensure_ascii=False)) # ensure_ascii=False để hiển thị tiếng Việt

# 3. Gửi yêu cầu đến API
try:
    pprint(f"\nĐang gửi yêu cầu tới {ASK_API_ENDPOINT}...")
    response = requests.post(ASK_API_ENDPOINT, json=payload, timeout=TIMEOUT)
    response.raise_for_status() # Ném ngoại lệ cho các mã trạng thái lỗi (4xx hoặc 5xx)

    result = response.json()

    # 4. Hiển thị kết quả
    pprint("\n--- KẾT QUẢ TỪ RAG API ---")
    pprint(f"Câu hỏi: {result.get('question', 'N/A')}")
    pprint(f"Tùy chọn đã hỏi: {result.get('options', 'N/A')}")
    pprint(f"Đáp án dự đoán: {result.get('predicted_option', 'N/A')}")
    pprint(f"Văn bản trả lời: {result.get('answer_text', 'N/A')}")
    pprint(f"Lý do: {result.get('reasoning', 'N/A')}")
    pprint(f"Độ tin cậy: {result.get('confidence', 'N/A')}")
    pprint(f"Chế độ tìm kiếm: {result.get('search_mode', 'N/A')}")
    pprint(f"Mô hình LLM: {result.get('model', 'N/A')}")
    pprint(f"Từ bộ nhớ đệm: {result.get('from_cache', False)}")
    pprint(f"Thời gian: {result.get('timing', {}).get('total_ms', 'N/A')}ms")

    sources = result.get('sources', [])
    if sources:
        print("\n--- NGUỒN TÀI LIỆU (Sources) ---")
        for i, source in enumerate(sources):
            pprint(f"  Source {i+1}:")
            pprint(f"    Document ID: {source.get('document_id', 'N/A')}")
            pprint(f"    Chunk ID: {source.get('chunk_id', 'N/A')}")
            pprint(f"    Section: {source.get('section_name', 'N/A')}")
            pprint(f"    Score: {source.get('score', 'N/A'):.4f}")
            pprint(f"    Preview: {source.get('preview', 'N/A')[:150]}...") # Giới hạn độ dài preview
            pprint("-" * 20)
    else:
        print("\n⚠️ Không có nguồn tài liệu nào được trả về.")

except requests.exceptions.RequestException as e:
    print(f"\n❌ Lỗi khi gọi API: {e}")
    if response is not None:
        print(f"Mã trạng thái HTTP: {response.status_code}")
        print(f"Phản hồi lỗi từ server: {response.text}")
except json.JSONDecodeError:
    print(f"\n❌ Lỗi khi phân tích phản hồi JSON. Phản hồi thô:\n{response.text}")
except Exception as e:
    print(f"\n❌ Đã xảy ra lỗi không mong muốn: {e}")